### Set up the panel for data to be merged in

In [1]:
clear
cd
// There are 13213 signal intersections
global intersect = 13213+1
global periods_per_year = 4
global obs = $periods_per_year*7*($intersect)

display $obs
set obs $obs



D:\Box Sync\Policy Evaluation\dofiles




369992

number of observations (_N) was 0, now 369,992


In [2]:
egen intersection_id = seq(), from(1) to($intersect)
sort intersection_id

egen quarter = seq(), from(1) to (4) by(intersection_id)
egen year = seq(), from(2012) to (2018) block(4) 

gen quarterly = yq(year,quarter)
format quarterly %tq

//These are outside of the range of our data
drop if quarter <= 2 & year == 2012
drop if quarter >= 4 & year == 2018








(26,428 observations deleted)

(13,214 observations deleted)


### Merge in Intersection data from "Signal intersection.ipynb"


In [3]:
// Intersection level data
mmerge intersection_id using "..\working_data\signal_intersection.dta", ///
type(n:1) ///
unmatched(using) ///
umatch(intersection_id)



-------------------------------------------------------------------------------
merge specs          |
       matching type | n:1
  mv's on match vars | none
  unmatched obs from | using
---------------------+---------------------------------------------------------
  master        file | <data in memory not named>
                 obs | 330350
                vars |      4
          match vars | intersection_id  (not a key)
  -------------------+---------------------------------------------------------
  using         file | ..\working_data\signal_intersection.dta
                 obs |  13213
                vars |     38
          match vars | intersection_id  (key)
---------------------+---------------------------------------------------------
variable intersection_id does not uniquely identify observations in the master data
result          file | <data in memory not named>
                 obs | 330325
                vars |     43  (including _merge)
         ------------+----

In [4]:
describe


Contains data
  obs:       330,325                          
 vars:            42                          
 size:   153,270,800                          
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
              storage   display    value
variable name   type    format     label      variable label
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
intersection_id int     %8.0g                 
quarter         byte    %8.0g                 
year            int     %8.0g                 
quarterly       int     %tq                   
index           int     %12.0g            

### Create borough dummies


In [5]:
// Create borough dummies
tab bname,m
gen bronx = (bname == "Bronx")
gen brooklyn = (bname == "Brooklyn")
gen manhattan = (bname == "Manhattan")
gen queens = (bname == "Queens")
gen statenisland = (bname == "Staten Island")



        bname |      Freq.     Percent        Cum.
--------------+-----------------------------------
        Bronx |     42,600       12.90       12.90
     Brooklyn |    117,400       35.54       48.44
    Manhattan |     71,775       21.73       70.17
       Queens |     82,800       25.07       95.23
Staten Island |     15,650        4.74       99.97
          nan |        100        0.03      100.00
--------------+-----------------------------------
        Total |    330,325      100.00







### Lion Street data 

In [6]:
// Lion Street data 
// Variables: Number_Tra Number_Par Number_Tot TrafDir distance_to_Street nearest_Street
rename Number_Tra num_traveling_lanes
rename Number_Par num_parking_lanes
rename Number_Tot num_total_lanes
rename TrafDir trafdir

In [7]:
replace num_traveling_lanes = "" if num_traveling_lanes == "None"
replace num_parking_lanes = "" if num_parking_lanes == "None"
replace num_total_lanes = "" if num_total_lanes == "None"
destring num_traveling_lanes num_parking_lanes num_total_lanes, replace


(40,250 real changes made)

(79,600 real changes made)

(40,275 real changes made)

num_traveling_lanes: all characters numeric; replaced as byte
(40250 missing values generated)
num_parking_lanes: all characters numeric; replaced as byte
(79600 missing values generated)
num_total_lanes: all characters numeric; replaced as byte
(40275 missing values generated)


In [8]:
replace trafdir = "1" if trafdir == "W"
replace trafdir = "2" if trafdir == "A"
replace trafdir = "3" if trafdir == "T"
replace trafdir = "4" if trafdir == "P"
replace trafdir = "" if trafdir == "None"
destring trafdir, replace

label define trafdir1 1 "With: One-way street, traffic flows with the segment's directionality" ///
2 "Against: One-way street, traffic flows from against the segment's directionality" ///
3 "Two-Way: Traffic flows in both directions" ///
4 "Pedestrian path: Non-vehicular"
label values trafdir trafdir1


(69,450 real changes made)

(66,325 real changes made)

(171,825 real changes made)

(1,975 real changes made)

(20,750 real changes made)

trafdir: all characters numeric; replaced as byte
(20750 missing values generated)




In [9]:
// Problem 1: Missing intersection data
// Solution 1: Inpute it as mean
codebook num_traveling_lanes
gen mi_number_travel_lanes = mi(num_traveling_lanes), after(num_traveling_lanes)
sum num_traveling_lanes
replace num_traveling_lanes = r(mean) if mi_number_travel_lanes == 1

codebook num_parking_lanes
gen mi_number_park_lanes = mi(num_parking_lanes), after(num_parking_lanes)
sum num_parking_lanes
replace num_parking_lanes = 0 if mi_number_park_lanes == 1



---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
num_traveling_lanes                                                                                                                                                                                                                                 (unlabeled)
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

                  type:  numeric (byte)

                 range:  [1,8]                        units:  1
         unique values:  7                        missing .:  40,250/330,325

            tabulation:  Freq.  Value
        

### Nearest Bike Route data

In [10]:
// Nearest Bike Route data
// Variables nearest_bikeroute distance_to_bikeroute
destring nearest_bikeroute,replace
mmerge nearest_bikeroute using "..\working_data\bike_routes_2263.dta", ///
type(n:1) ///
unmatched(master) ///
umatch(objectid_1) ///
ukeep(date_instd tf_facilit)


nearest_bikeroute already numeric; no replace


-------------------------------------------------------------------------------
merge specs          |
       matching type | n:1
  mv's on match vars | none
  unmatched obs from | master
---------------------+---------------------------------------------------------
  master        file | <data in memory not named>
                 obs | 330325
                vars |     48
          match vars | nearest_bikeroute  (not a key)
  -------------------+---------------------------------------------------------
  using         file | ..\working_data\bike_routes_2263.dta
                 obs |  14980
                vars |      3  (selection via udrop/ukeep)
          match vars | objectid_1  (key)
---------------------+---------------------------------------------------------
variable nearest_bikeroute does not uniquely identify observations in the master data
result          file | <data in memory not named>
                 obs | 330325
   

In [11]:
gen bike_route_install_dt = date(date_instd, "YMD"), after(date_instd)
format bike_route_install_dt %td
gen bike_route_install_mt = mofd(bike_route_install_dt)
format bike_route_install_mt %tm
gen bike_route_install_qt = qofd(bike_route_install_dt)
format bike_route_install_qt %tq
drop date_instd

In [12]:
// Determine if collision is on bike route
sum distance_to_bikeroute, detail
sum distance_to_bikeroute if distance_to_bikeroute < r(p50) , detail

gen bike_route_ever = distance_to_bikeroute <= 3
tab bike_route_ever
label variable bike_route_ever "Indicates that bike route passes through this intersection (<3 ft)"


// Timevarient bike route indicator
gen bike_route_tv = 0 
replace bike_route_tv = 1 if quarterly >= bike_route_install_qt & bike_route_ever == 1

label variable bike_route_tv "Indicates that bike route passes through this intersection timevarient(<3 ft)"



                    distance_to_bikeroute
-------------------------------------------------------------
      Percentiles      Smallest
 1%            0              0
 5%            0              0
10%            1              0       Obs             330,325
25%            3              0       Sum of Wgt.     330,325

50%          537                      Mean           1023.112
                        Largest       Std. Dev.      1365.944
75%         1416           8907
90%         2871           8907       Variance        1865802
95%         4039           8907       Skewness        2.00231
99%         6101           8907       Kurtosis       7.320251


                    distance_to_bikeroute
-------------------------------------------------------------
      Percentiles      Smallest
 1%            0              0
 5%            0              0
10%            0              0       Obs             165,025
25%            1              0       Sum of Wgt.     165,025

50% 

## Truck Route

In [13]:
// Truck route
// Determine if collision is on truck route
sum distance_to_truckroute, detail
sum distance_to_truckroute if distance_to_truckroute < r(p50) , detail
sum distance_to_truckroute if distance_to_truckroute < r(p90) , detail
gen truck_route = distance_to_truckroute <= 3
tab truck_route

label variable truck_route "Indicates that truck route passes through this intersection (<3 ft)"




                   distance_to_truckroute
-------------------------------------------------------------
      Percentiles      Smallest
 1%            2              0
 5%            2              0
10%            2              0       Obs             330,325
25%            3              0       Sum of Wgt.     330,325

50%          263                      Mean           551.0966
                        Largest       Std. Dev.      753.6419
75%          827           9622
90%         1569           9622       Variance       567976.1
95%         2118           9622       Skewness       2.078691
99%         3125           9622       Kurtosis       10.19186


                   distance_to_truckroute
-------------------------------------------------------------
      Percentiles      Smallest
 1%            1              0
 5%            2              0
10%            2              0       Obs             164,775
25%            2              0       Sum of Wgt.     164,775

50% 

### Left turn intervention

In [14]:
// Left turn calming interventions information
// Variables treatment_ completion nearest_LTC distance_to_LTC
rename treatment_ left_turn_treatment

gen left_turn_install_dt = date(completion, "YMD"), after(completion)
format left_turn_install_dt %td
gen int left_turn_install_mt = mofd(left_turn_install_dt), after(left_turn_install_dt)
format left_turn_install_mt %tm
gen int left_turn_install_qt = qofd(left_turn_install_dt), after(left_turn_install_dt)
format left_turn_install_qt %tq
drop completion

In [15]:
// Matching intersection to closest Left turn calming interventions
egen double left_turn_min = min(distance_to_LTC), by(nearest_LTC)
gen flag_left_turn_ever = .
replace flag_left_turn_ever = 1 if left_turn_min == distance_to_LTC
replace flag_left_turn_ever = 0 if mi(flag_left_turn_ever)

gen flag_left_turn = flag_left_turn_ever
replace flag_left_turn = 0 if flag_left_turn == 1 & quarterly < left_turn_install_qt



(330,325 missing values generated)

(5,800 real changes made)

(324,525 real changes made)


(4,359 real changes made)


### Street improvement project

In [16]:
// Street Improv Project interventions information
// Variable DateComplt SIPProjTyp distance_to_StImpro nearest_StImpro
rename SIPProjTyp street_improv_treatment

gen street_improv_install_dt = date(DateComplt, "YMD"), after(DateComplt)
format street_improv_install_dt %td
gen int street_improv_install_mt = mofd(street_improv_install_dt), after(street_improv_install_dt)
format street_improv_install_mt %tm
gen int street_improv_install_qt = qofd(street_improv_install_dt), after(street_improv_install_dt)
format street_improv_install_qt %tq
drop DateComplt

In [17]:
// Matching intersection to closest street improvment interventions
egen double street_improv_min = min(distance_to_StImpro), by(nearest_StImpro)
gen flag_street_improv_ever = .
replace flag_street_improv_ever = 1 if street_improv_min == distance_to_StImpro
replace flag_street_improv_ever = 0 if mi(flag_street_improv_ever)

gen flag_street_improv = flag_street_improv_ever
replace flag_street_improv = 0 if flag_street_improv == 1 & quarterly < street_improv_install_qt



(330,325 missing values generated)

(9,125 real changes made)

(321,200 real changes made)


(4,592 real changes made)


### Leading Pedestrian Interval Signal

In [18]:
// LPIS information
// Variable distance_to_LPIS nearest_LPIS date_insta
gen LPIS_install_dt = date(date_insta, "YMD")
format LPIS_install_dt %td
gen int LPIS_install_mt = mofd(LPIS_install_dt), after(LPIS_install_dt)
format LPIS_install_mt %tm
gen int LPIS_install_qt = qofd(LPIS_install_dt), after(LPIS_install_mt)
format LPIS_install_qt %tq 
gen LPIS_install_yr = year(LPIS_install_dt), after(LPIS_install_qt)
drop date_insta

In [19]:
// Matching intersection to closest LPIS intersection
egen double LPIS_min = min(distance_to_LPIS), by(nearest_LPIS)
gen flag_LPIS_ever = .
replace flag_LPIS_ever = 1 if LPIS_min == distance_to_LPIS
replace flag_LPIS_ever = 0 if mi(flag_LPIS_ever)

gen flag_LPIS = flag_LPIS_ever



(330,325 missing values generated)

(72,875 real changes made)

(257,450 real changes made)



In [20]:
// Problem2: Missing Install date on some LPIS IDs
sum LPIS_install_qt, detail format
codebook LPIS_install_qt 
tab nearest_LPIS if mi(LPIS_install_qt) & flag_LPIS == 1
tab intersection_id if mi(LPIS_install_qt) & flag_LPIS == 1
sum distance_to_LPIS if mi(LPIS_install_qt)

// Solution2: Drop these intersections
tab flag_LPIS if mi(LPIS_install_qt)
drop if mi(LPIS_install_qt) & flag_LPIS == 1
//0 observations deleted

// Replace with indicator with 0 if quarter was before LPIS installation
replace flag_LPIS = 0 if flag_LPIS == 1 & quarterly < LPIS_install_qt
// 47,593 real changes made



                       LPIS_install_qt
-------------------------------------------------------------
      Percentiles      Smallest
 1%    1899q4      1899q4
 5%    2004q1      1899q4
10%    2013q2      1899q4       Obs             330,325
25%    2015q4      1899q4       Sum of Wgt.     330,325

50%    2016q3                      Mean          2014q2
                        Largest       Std. Dev.       51.7974
75%    2017q3      2018q3
90%    2018q2      2018q3       Variance       2682.971
95%    2018q3      2018q3       Skewness      -7.818992
99%    2018q3      2018q3       Kurtosis       68.19259


---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
LPIS_install_qt                                                                                                                   

In [21]:
// Problem3: Certain LPIS junctions have been installed 
// before the collisions datasets begin
// Removing junctions that cannot be analyzed because LPIS was done before 01jul2012
sum quarterly, detail format
sum LPIS_install_qt, detail format

// Solution3: Drop these observations 
tab flag_LPIS if LPIS_install_qt < tq(2013q1)
tab intersection_id if LPIS_install_qt < tq(2013q1) & flag_LPIS_ever == 1
drop if LPIS_install_qt < tq(2013q1) & flag_LPIS_ever == 1
* 5,650 observations deleted



                          quarterly
-------------------------------------------------------------
      Percentiles      Smallest
 1%    2012q3      2012q3
 5%    2012q4      2012q3
10%    2013q1      2012q3       Obs             330,325
25%    2014q1      2012q3       Sum of Wgt.     330,325

50%    2015q3                      Mean          2015q3
                        Largest       Std. Dev.      7.211113
75%    2017q1      2018q3
90%    2018q1      2018q3       Variance       52.00016
95%    2018q2      2018q3       Skewness              0
99%    2018q3      2018q3       Kurtosis       1.796154


                       LPIS_install_qt
-------------------------------------------------------------
      Percentiles      Smallest
 1%    1899q4      1899q4
 5%    2004q1      1899q4
10%    2013q2      1899q4       Obs             330,325
25%    2015q4      1899q4       Sum of Wgt.     330,325

50%    2016q3                      Mean          2014q2
                        Largest    

       3005 |         25        0.44       60.18
       3140 |         25        0.44       60.62
       3455 |         25        0.44       61.06
       3561 |         25        0.44       61.50
       4090 |         25        0.44       61.95
       4340 |         25        0.44       62.39
       4701 |         25        0.44       62.83
       4735 |         25        0.44       63.27
       5042 |         25        0.44       63.72
       5054 |         25        0.44       64.16
       5276 |         25        0.44       64.60
       5278 |         25        0.44       65.04
       5285 |         25        0.44       65.49
       5314 |         25        0.44       65.93
       5345 |         25        0.44       66.37
       5346 |         25        0.44       66.81
       5348 |         25        0.44       67.26
       5363 |         25        0.44       67.70
       5385 |         25        0.44       68.14
       5398 |         25        0.44       68.58
       5418 |       

### Checking that panel is strongly balanced

In [22]:
duplicates tag intersection_id, gen(dup2)
tab dup2
drop dup2



Duplicates in terms of intersection_id


       dup2 |      Freq.     Percent        Cum.
------------+-----------------------------------
         24 |    324,675      100.00      100.00
------------+-----------------------------------
      Total |    324,675      100.00



In [23]:
label variable flag_LPIS "Indicates the intersection at the time they became an LPIS intervention"
label variable flag_LPIS_ever "Indicates the intersection if they ever received LPIS intervention"

### Determine if intersection is within 2000ft of LPIS intersection

In [24]:
mmerge intersection_id using "..\working_data\intersection_2000ft_LPIS_reshaped.dta", ///
type(n:1) ///
unmatched(master) ///
umatch(intersection_id)



-------------------------------------------------------------------------------
merge specs          |
       matching type | n:1
  mv's on match vars | none
  unmatched obs from | master
---------------------+---------------------------------------------------------
  master        file | <data in memory not named>
                 obs | 324675
                vars |     71
          match vars | intersection_id  (not a key)
  -------------------+---------------------------------------------------------
  using         file | ..\working_data\intersection_2000ft_LPIS_reshaped.dta
                 obs |  11762
                vars |     54
          match vars | intersection_id  (key)
---------------------+---------------------------------------------------------
variable intersection_id does not uniquely identify observations in the master data
result          file | <data in memory not named>
                 obs | 324675
                vars |    126  (including _merge)
         --

### Safe streets for Seniors 

In [25]:
codebook Name


---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Name                                                                                                                                                                                                                                                (unlabeled)
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

                  type:  string (str26)

         unique values:  42                       missing "":  0/324,675

              examples:  "nan"
                         "nan"
                         "nan"
                      

In [26]:
tab Name


                      Name |      Freq.     Percent        Cum.
---------------------------+-----------------------------------
                   Astoria |        775        0.24        0.24
                Bath Beach |        975        0.30        0.54
                 Bay Ridge |      1,700        0.52        1.06
        Bedford Stuyvesant |      4,600        1.42        2.48
               Bensonhurst |      2,475        0.76        3.24
              Borough Park |      5,000        1.54        4.78
            Brighton Beach |        225        0.07        4.85
               Brownsville |      5,100        1.57        6.42
        Chelsea/9th Avenue |      1,900        0.59        7.01
                 Chinatown |        475        0.15        7.15
           Corona Elmhurst |      7,125        2.19        9.35
            East Concourse |        450        0.14        9.49
             East Flatbush |        600        0.18        9.67
               East Harlem |      2,625

In [27]:
gen flag_seniors = (strpos(Name, "nan") == 0)

In [28]:
tab flag_seniors


flag_senior |
          s |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |    260,275       80.16       80.16
          1 |     64,400       19.84      100.00
------------+-----------------------------------
      Total |    324,675      100.00


In [29]:
tab flag_seniors flag_LPIS


           |     Indicates the
           |  intersection at the
           |  time they became an
flag_senio |   LPIS intervention
        rs |         0          1 |     Total
-----------+----------------------+----------
         0 |   246,705     13,570 |   260,275 
         1 |    58,336      6,064 |    64,400 
-----------+----------------------+----------
     Total |   305,041     19,634 |   324,675 


### Priority Intersection

In [30]:
sum distance_to_p*
gen flag_priorityinters = (distance_to_priorityinters < 10)



    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
distance_t~s |    324,675    2603.942    2534.222          0      28481



In [31]:
tab flag_priorityinters flag_LPIS


           |     Indicates the
           |  intersection at the
           |  time they became an
flag_prior |   LPIS intervention
 ityinters |         0          1 |     Total
-----------+----------------------+----------
         0 |   300,649     18,551 |   319,200 
         1 |     4,392      1,083 |     5,475 
-----------+----------------------+----------
     Total |   305,041     19,634 |   324,675 


### School intersection/crossing

In [32]:
sum distance_to_Sch,detail
gen flag_school = (distance_to_Sch < 200)



                       distance_to_Sch
-------------------------------------------------------------
      Percentiles      Smallest
 1%          125             17
 5%          197             17
10%          246             17       Obs             324,675
25%          425             17       Sum of Wgt.     324,675

50%          692                      Mean           822.7985
                        Largest       Std. Dev.      612.1056
75%         1055          10685
90%         1511          10685       Variance       374673.3
95%         1872          10685       Skewness       3.568631
99%         2871          10685       Kurtosis        35.5307



In [33]:
tab flag_school flag_LPIS 


           |     Indicates the
           |  intersection at the
           |  time they became an
flag_schoo |   LPIS intervention
         l |         0          1 |     Total
-----------+----------------------+----------
         0 |   289,690     18,085 |   307,775 
         1 |    15,351      1,549 |    16,900 
-----------+----------------------+----------
     Total |   305,041     19,634 |   324,675 


### Merge outcomes data

In [34]:
// Outcomes data
mmerge quarter year intersection_id using "..\working_data\collision_quarterly.dta", ///
type(1:1) ///
unmatched(master) 
// Reason that we did not match every obs from using
// is that we droped intersections in the 
// 1.Intersection level data merge process
// 2. Problem2 Certain LPIS junctions have been installed 
// before the collisions datasets begin


-------------------------------------------------------------------------------
merge specs          |
       matching type | 1:1
  mv's on match vars | none
  unmatched obs from | master
---------------------+---------------------------------------------------------
  master        file | <data in memory not named>
                 obs | 324675
                vars |    127
          match vars | quarter year intersection_id  (key)
  -------------------+---------------------------------------------------------
  using         file | ..\working_data\collision_quarterly.dta
                 obs | 195341
                vars |     30
          match vars | quarter year intersection_id  (key)
---------------------+---------------------------------------------------------
result          file | <data in memory not named>
                 obs | 324675
                vars |    156  (including _merge)
         ------------+---------------------------------------------------------
          

### Fill in missing information

In [35]:
// Fill in '0' for intersection month year that did not have collisions
replace personsinjured = 0 if _merge == 1
replace personskilled = 0 if _merge == 1
replace pedestriansinjured = 0 if _merge == 1
replace pedestrianskilled = 0 if _merge == 1
replace cyclistinjured = 0 if _merge == 1
replace cyclistkilled = 0 if _merge == 1
replace motoristinjured = 0 if _merge == 1
replace motoristkilled = 0 if _merge == 1
replace collision_count = 0 if mi(collision_count)

replace latenight_personsinjured = 0 if _merge == 1
replace latenight_personskilled = 0 if _merge == 1
replace latenight_pedestriansinjured = 0 if _merge == 1
replace latenight_pedestrianskilled = 0 if _merge == 1
replace latenight_cyclistinjured = 0 if _merge == 1
replace latenight_cyclistkilled = 0 if _merge == 1
replace latenight_motoristinjured = 0 if _merge == 1
replace latenight_motoristkilled = 0 if _merge == 1
replace latenight_collision_count = 0 if mi(latenight_collision_count)

replace day_personsinjured = 0 if _merge == 1
replace day_personskilled = 0 if _merge == 1
replace day_pedestriansinjured = 0 if _merge == 1
replace day_pedestrianskilled = 0 if _merge == 1
replace day_cyclistinjured = 0 if _merge == 1
replace day_cyclistkilled = 0 if _merge == 1
replace day_motoristinjured = 0 if _merge == 1
replace day_motoristkilled = 0 if _merge == 1
replace day_collision_count = 0 if mi(day_collision_count)

gen flag_collision = collision_count > 0
gen latenight_flag_collision = latenight_collision_count > 0
gen day_flag_collision = day_collision_count > 0


label variable personsinjured "No. of persons injured during that month"
label variable personskilled "No. of persons killed during that month"
label variable pedestriansinjured "No. of pedestrians injured during that month"
label variable pedestrianskilled "No. of pedestrians killed during that month"
label variable cyclistinjured "No. of cyclist injured during that month"
label variable cyclistkilled "No. of cyclist killed during that month"
label variable motoristinjured "No. of motorist injured during that month"
label variable motoristkilled "No. of motorist killed during that month"
label variable collision_count "No. of collisions occured during that month"
label variable flag_collision "At least one collision occurred that month"

label variable latenight_personsinjured "No. of persons injured during that month between 11pm - 5am"
label variable latenight_personskilled "No. of persons killed during that month between 11pm - 5am"
label variable latenight_pedestriansinjured "No. of pedestrians injured during that month between 11pm - 5am"
label variable latenight_pedestrianskilled "No. of pedestrians killed during that month between 11pm - 5am"
label variable latenight_cyclistinjured "No. of cyclist injured during that month between 11pm - 5am"
label variable latenight_cyclistkilled "No. of cyclist killed during that month between 11pm - 5am"
label variable latenight_motoristinjured "No. of motorist injured during that month between 11pm - 5am"
label variable latenight_motoristkilled "No. of motorist killed during that month between 11pm - 5am"
label variable latenight_collision_count "No. of collisions occured during that month between 11pm - 5am"
label variable latenight_flag_collision " At least one collision occurred that month between 11pm - 5am"

label variable day_personsinjured "No. of persons injured during that month between 5am - 11pm"
label variable day_personskilled "No. of persons killed during that month between 5am - 11pm"
label variable day_pedestriansinjured "No. of pedestrians injured during that month between 5am - 11pm"
label variable day_pedestrianskilled "No. of pedestrians killed during that month between 5am - 11pm"
label variable day_cyclistinjured "No. of cyclist injured during that month between 5am - 11pm"
label variable day_cyclistkilled "No. of cyclist killed during that month between 5am - 11pm"
label variable day_motoristinjured "No. of motorist injured during that month between 5am - 11pm"
label variable day_motoristkilled "No. of motorist killed during that month between 5am - 11pm"
label variable day_collision_count "No. of collisions occured during that month between 5am - 11pm"
label variable day_flag_collision " At least one collision occurred that month between 5am - 11pm"

drop _merge


(133,939 real changes made)

(133,939 real changes made)

(133,939 real changes made)

(133,939 real changes made)

(133,939 real changes made)

(133,939 real changes made)

(133,939 real changes made)

(133,939 real changes made)

(133,939 real changes made)

(133,939 real changes made)

(133,939 real changes made)

(133,939 real changes made)

(133,939 real changes made)

(133,939 real changes made)

(133,939 real changes made)

(133,939 real changes made)

(133,939 real changes made)

(133,939 real changes made)

(133,939 real changes made)

(133,939 real changes made)

(133,939 real changes made)

(133,939 real changes made)

(133,939 real changes made)

(133,939 real changes made)

(133,939 real changes made)

(133,939 real changes made)

(133,939 real changes made)




































In [36]:
save "..\working_data\analytical_file_panel_qt.dta",replace

file ..\working_data\analytical_file_panel_qt.dta saved
